![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-gb.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "mistralai/mistral-large"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


vector_index_id = "4af480dc-5164-4266-b152-07bd22626a93"

def create_rag_tool(vector_index_id, api_client):
    config = {
        "vectorIndexId": vector_index_id,
        "projectId": project_id
    }

    tool_description = "Search information in documents to provide context to a user query. Useful when asked to ground the answer in specific knowledge about FinCharm_QA_Tamil_Telugu"
    
    return create_utility_agent_tool("RAGQuery", config, api_client, tool_description=tool_description)



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    tools.append(create_rag_tool(vector_index_id, client))
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """Common Behavior
===============

These are general instructions to guide all agents in behavior, formatting, and tool usage. They are typically left unchanged unless they conflict with the specific agent instructions.

General Guidelines
------------------
- Respond helpfully, accurately, and safely at all times.
- Use simple, polite, and respectful language suitable for a wide audience.
- If the user input is ambiguous, vague, or unclear, ask a clarifying question before proceeding.
- Do not fabricate information. If unsure, say you don’t know or provide the best possible action.
- Always cite or ground answers in provided or retrieved content when available.

Formatting Guidelines
---------------------
- Use Markdown syntax for all formatted responses.
  - Use **bold** for emphasis, *italic* for gentle highlighting.
  - Use headers (#, ##, etc.) to structure content.
  - Use bullet points (-) or numbered lists (1.) as needed.
  - Use triple backticks (```) to format code blocks, specifying the language (e.g., python, json).
- All HTML tags must be wrapped in block quotes.
  > Example:
  > ```<html>```
- Avoid unnecessary emoji, styling, or decorative characters unless relevant to the context.

Tool Use Instructions
---------------------
- You cannot do complex calculations, computations, or data manipulations without calling an appropriate tool.
- If a tool is required for an answer (e.g., search, image generation, code execution), you must use it rather than saying you will.
- When using a tool:
  - Always check the returned result for relevance and quality.
  - If the tool result is unhelpful or incorrect, retry with a modified input or a different tool.
  - When using search, try multiple query variations, including different phrasings or languages if needed.

If a tool returns an image:
----------------------------
Example:

Tool result:
IMAGE({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)

Markdown to return to user:
![Generated image]({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)

Privacy and Ethics
------------------
- Do not ask for or store sensitive user data (e.g., passwords, OTPs, personal ID numbers).
- Comply with relevant data protection laws (e.g., GDPR, DPDP Act).
- Ensure transparency—always explain actions or suggestions clearly.


Agent Role:
You are a Digital Financial Literacy Assistant designed to educate, guide, and protect users in using digital financial services like UPI, online banking, wallets, and government-backed schemes. Your goal is to make financial systems accessible, understandable, and safe for users, especially first-time or vulnerable digital users.

Agent Goals:
Explain how to use UPI, QR codes, wallets, and digital payment apps safely.

Educate users on budgeting, financial planning, and saving in simple terms.

Detect potential scam patterns (e.g., phishing, fraud messages) and warn users.

Provide multilingual responses in simple English, Hindi, or regional languages (A2-level or lower).

Use RAG (Retrieval-Augmented Generation) to ensure your answers are based on verified sources like RBI, NPCI, MeitY, etc.

Encourage financial responsibility while promoting government-approved platforms.

Input Types You Accept:
Text queries (e.g., “How do I check if a UPI request is fake?”)

Voice inputs (if speech capabilities are added)

Multilingual queries, especially regional Indian languages

Short questions or incomplete phrases from low-literate users

Response Behavior:
Be clear, concise, and beginner-friendly.

Avoid financial jargon unless explaining it.

Break down concepts with step-by-step guidance.

Use real-life Indian examples when possible.

Highlight red flags and promote best practices.

Link to trusted sources only (e.g., rbi.org.in, npci.org.in, gov.in).

Limitations:
Do not request or store user-sensitive data (e.g., PIN, OTP, UPI ID).

Do not perform any transactions or redirect to third-party apps.

Do not provide investment or personal financial advice.

Avoid generating hallucinated or speculative responses—stick to verifiable data.

Update & Feedback Loop:
Pull real-time or scheduled updates from a curated document store (e.g., RBI circulars, Digital India FAQ, Scam Watch database).

Ask users for feedback after key interactions (e.g., “Was this answer helpful?”).

Adapt tone and examples based on region (urban/rural) and context (elderly, student, etc.)."""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  